In [37]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets.mnist import MNIST
from torch.utils.data import DataLoader
from collections import OrderedDict
from torchsummary import summary
use_cuda = torch.cuda.is_available()
print(use_cuda)
%matplotlib inline
from foolbox.criteria import Misclassification
import foolbox
import torchvision.models as models
import numpy as np
from torch.autograd import Variable
import torch.utils.data as utils
from torch.utils.data import Dataset, DataLoader

True


In [38]:
# train_orig = np.load('./mnist_pure/x_train.npy')
# train_orig_labels = np.load('./mnist_pure/y_train.npy')
# train_orig_labels = train_orig_labels[:30000]
# #print(train_orig_labels[0])
# train_orig = train_orig[:30000]
# train_adv = np.load('./attack_1/adv_imgs_train.npy')
# train_adv_labels = np.load('./attack_1/adv_labels_train.npy')
# train_adv_labels = train_adv_labels[:30000]
# #print(train_adv_labels[0])
# train_adv = train_adv[:30000]
# x_train = np.concatenate([train_orig, train_adv], axis=0)
# y_train = np.concatenate([train_orig_labels, train_adv_labels], axis=0)
# x_train = x_train.reshape(60000,1,28,28)
# #y_train = y_train.reshape(60000,1,28,28)
# print(x_train.shape)
# print(y_train.shape)

# test_orig = np.load('./mnist_pure/x_test.npy')
# test_orig_labels = np.load('./mnist_pure/y_test.npy')
# test_orig_labels = test_orig_labels[:5000]
# test_orig = test_orig[:5000]
# test_adv = np.load('./attack_1/adv_imgs_test.npy')
# test_adv_labels = np.load('./attack_1/adv_labels_test.npy')
# test_adv = test_adv[:5000]
# test_adv_labels = test_adv_labels[:5000]
# x_test = np.concatenate([test_orig, test_adv], axis=0)
# y_test = np.concatenate([test_orig_labels, test_adv_labels], axis=0)
# #print(y_test[0].shape)
# x_test = x_test.reshape(10000,1,28,28)
# #y_test = y_test.reshape(10000,1,28,28)
# print(x_test.shape)
# print(y_test.shape)
# #print(y_test[0])


In [39]:
def random(array_imgs, array_labels, value, length):
    rand_idx = np.random.randint(value, size=length)
    new_arr = []
    new_arr_labels =[]
    for i in range(0,length):
        new_arr.append(array_imgs[rand_idx[i]])
        new_arr_labels.append(array_labels[rand_idx[i]])
    new_arr = np.asarray(new_arr)
    new_arr_labels = np.asarray(new_arr_labels)
    return new_arr, new_arr_labels

In [40]:
#Training
train_value = 30000
ori_train_length = 30000
adv_train_length = 0

#Testing
test_value = 5000
ori_test_length = 5000
adv_test_length = 0


#Training

train_orig = np.load('./mnist_pure/x_train.npy')
train_orig_labels = np.load('./mnist_pure/y_train.npy')
train_orig, train_orig_labels = random(train_orig, train_orig_labels, value=train_value, length=ori_train_length)


train_adv = np.load('./attack_1/adv_imgs_train.npy')
train_adv_labels = np.load('./attack_1/adv_labels_train.npy')
train_adv, train_adv_labels = random(train_adv,train_adv_labels, value=train_value, length=adv_train_length)
y_train = np.concatenate([train_orig_labels, train_adv_labels], axis=0)
x_train = np.concatenate([train_orig, train_adv], axis=0)
x_train = x_train.reshape(60000,1,28,28)
print(x_train.shape)
print(y_train.shape)



test_orig = np.load('./mnist_pure/x_test.npy')
test_orig_labels = np.load('./mnist_pure/y_test.npy')
test_orig, test_orig_labels = random(test_orig, test_orig_labels, value=test_value, length=ori_test_length)


test_adv = np.load('./attack_1/adv_imgs_test.npy')
test_adv_labels = np.load('./attack_1/adv_labels_test.npy')
test_adv, test_adv_labels = random(test_adv, test_adv_labels, value=test_value, length=adv_test_length)
# y_test = np.concatenate([test_orig_labels, test_adv_labels], axis=0)
# x_test = np.concatenate([test_orig, test_adv], axis=0)
# x_test = x_test.reshape(10000,1,28,28)


print(x_test.shape)
print(y_test.shape)

(60000, 1, 28, 28)
(60000,)
(10000, 1, 28, 28)
(10000,)


In [41]:
class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

#trans = transforms.Compose([transforms.ToTensor()])
batch_size = 64

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True)

print ('==>>> total trainning batch number: {}'.format(len(train_loader)))
print ('==>>> total testing batch number: {}'.format(len(test_loader)))

dataiter = iter(train_loader)
img, label = dataiter.next()

==>>> total trainning batch number: 938
==>>> total testing batch number: 157


In [42]:
# To Train Original MNIST Dataset

# trans = transforms.Compose([transforms.ToTensor()])
# train_set = MNIST('./data', train=True, transform=trans, download=True)
# test_set = MNIST('./data', train=False, transform=trans, download=True)

# # train_set = MNIST('./data', train=True, download=True)
# # test_set = MNIST('./data', train=False, download=True)


# batch_size = 64

# train_loader = torch.utils.data.DataLoader(
#                  dataset=train_set,
#                  batch_size=batch_size,
#                  shuffle=True)
# test_loader = torch.utils.data.DataLoader(
#                 dataset=test_set,
#                 batch_size=batch_size,
#                 shuffle=False)

# print ('==>>> total trainning batch number: {}'.format(len(train_loader)))
# print ('==>>> total testing batch number: {}'.format(len(test_loader)))

In [43]:
class LeNet5(nn.Module):          
     
    def __init__(self):     
        super(LeNet5, self).__init__()
        # Convolution (In LeNet-5, 32x32 images are given 
        # as input. Hence padding of 2 is done below)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, 
                                     kernel_size=5, stride=1, padding=2)
        self.max_pool_1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, 
                                     kernel_size=5, stride=1, padding=2)
        self.max_pool_2 = nn.MaxPool2d(kernel_size=2, stride=2) 
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, 
                                     kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(7*7*120, 120)
        # convert matrix with 16*5*5 (= 400) features to a matrix of 120 features (columns)
        self.fc2 = nn.Linear(120, 84)       
        # convert matrix with 120 features to a matrix of 84 features (columns)
        self.fc3 = nn.Linear(84, 10)        
        # convert matrix with 84 features to a matrix of 10 features (columns)

            
    def forward(self, x):
        # convolve, then perform ReLU non-linearity
        x = F.relu(self.conv1(x))  
        # max-pooling with 2x2 grid 
        x = self.max_pool_1(x) 
        # Conv2 + ReLU
        x = F.relu(self.conv2(x))
        # max-pooling with 2x2 grid
        x = self.max_pool_2(x)
        # Conv3 + ReLU
        x = F.relu(self.conv3(x))
        x = x.view(-1, 7*7*120)
        # FC-1, then perform ReLU non-linearity
        x = F.relu(self.fc1(x))
        # FC-2, then perform ReLU non-linearity
        x = F.relu(self.fc2(x))
        # FC-3
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)

In [44]:
model = LeNet5()
if use_cuda:
    model.cuda()

summary(model, (1, 28, 28))
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# criterion = nn.CrossEntropyLoss()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 14, 14]           2,416
         MaxPool2d-4             [-1, 16, 7, 7]               0
            Conv2d-5            [-1, 120, 7, 7]          48,120
            Linear-6                  [-1, 120]         705,720
            Linear-7                   [-1, 84]          10,164
            Linear-8                   [-1, 10]             850
Total params: 767,426
Trainable params: 767,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.12
Params size (MB): 2.93
Estimated Total Size (MB): 3.05
----------------------------------------------------------------


In [45]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda==True:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. 
        # It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % 4000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))
            

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda==True:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad(): # volatile was removed and now 
            # has no effect. Use `with torch.no_grad():` instead.
            data= Variable(data)
        target = Variable(target)
        output = model(data)
        # sum up batch loss # size_average and reduce args will 
        # be deprecated, please use reduction='sum' instead.
        test_loss += F.nll_loss(output, target, reduction='sum').data 
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1] 
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



epochs = 30

for epoch in range(1, epochs + 1):
    train(epoch)
    test()
torch.save(model.state_dict(), './models/lenet5_mnist_0.5.pt')
print("Model Saved Successfully . . .")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.289282

Test set: Average loss: 0.1509, Accuracy: 9525/10000 (95.00%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.139179

Test set: Average loss: 0.0675, Accuracy: 9783/10000 (97.00%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.082481

Test set: Average loss: 0.0513, Accuracy: 9828/10000 (98.00%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.029644

Test set: Average loss: 0.0530, Accuracy: 9837/10000 (98.00%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.003122

Test set: Average loss: 0.0497, Accuracy: 9849/10000 (98.00%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.043508

Test set: Average loss: 0.0582, Accuracy: 9828/10000 (98.00%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.103520

Test set: Average loss: 0.0592, Accuracy: 9841/10000 (98.00%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.001338

Test set: Average loss: 0.0497, Accuracy: 9871/10000 (98.00%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.002713

Test set: Average loss: 0.0623, Accuracy: 9849/10000 (98.00%)

Train Epoch: 10 [0/